# OCR文字识别训练

In [ ]:
import sagemaker
from sagemaker import get_execution_role
sagemaker_session = sagemaker.Session()

#FIXME: 修改自己的S3 bucket名称
bucket = 'YOUR_BUCKET'
prefix = 'sagemaker-ocr-chinese'
role = get_execution_role()
# 如果在自建EC2无法获取role， 可以手动复制 role
#  role = arn:aws-cn:iam::账户id:role/service-role/AmazonSageMaker-ExecutionRole-20200430T123312

target_s3_uri = 's3://{}/{}/'.format(bucket, prefix)

## 准备训练数据

#### 一 使用Demo数据 



In [ ]:
!aws s3 sync s3://dikers-public/sagemaker-ocr-chinese/ $target_s3_uri


#### 二  自己生成数据  


*  第一步  生成小图片    [参考代码](https://github.com/dikers/ocr-text-renderer)



```
# 生成图片和  label文件
# label 文件格式  前面是图片的路径， 后面是对应的gt
00000000.jpg F六G七H八I九J十
00000001.jpg e六f七g八h九i十
00000002.jpg W千X一Y二Z三?!
00000003.jpg t七u八v九w十x百
00000004.jpg 四P五Q六R七S八T
00000005.jpg Y二Z三?!@#%
00000006.jpg d五e六f七g八h九
00000007.jpg ,.A一B二C三D四
00000008.jpg p三q四r五s六t七
00000009.jpg 六t七u八v九w十x
```

*  第二步  请文件划分成 train.txt  valid.txt

```
head -n 10000 labels.txt > train.txt

tail -n 1000 labels.txt  >  valid.txt
```


* 第三步  将图片转换成mdb格式的文件


```
# 运行脚本
cd data_generate
sh create-lmdb.sh

```


```
# 修改脚本的路径
python3 create_lmdb_dataset.py --inputPath images_path/ \
--gtFile valid.txt \
--outputPath ./output/valid

python3 create_lmdb_dataset.py --inputPath images_path/  \
--gtFile train.txt \
--outputPath ./output/train
```


*  第四步 上传数据到S3

In [ ]:
"""
inputs = sagemaker_session.upload_data(path='文件路径', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))
"""

可以包含多个训练数据， 本地数据格式如下： 
```
.
├── train
│   ├── db1
│   │   ├── data.mdb
│   │   └── lock.mdb
│   └── db2
│       ├── data.mdb
│       └── lock.mdb
└── valid
    ├── db1
    │   ├── data.mdb
    │   └── lock.mdb
    └── db2
        ├── data.mdb
        └── lock.mdb

```

对应服务器的路径
```
'train_data': '/opt/ml/input/data/training/train',
'valid_data': '/opt/ml/input/data/training/valid',

'select_data': 'db1-db2',    # 训练数据的名称
'batch_ratio': '0.5-0.5',    # 训练数据对应的比率

```

上传的到S3的路径   
```
   s3://YOUR_BUCKET/sagemaker-ocr-chinese/
                        ├── train
                        │   ├── db1
                        │   │   ├── data.mdb
                        │   │   └── lock.mdb
                        │   └── db2
                        │       ├── data.mdb
                        │       └── lock.mdb
                        └── valid
                            ├── db1
                            │   ├── data.mdb
                            │   └── lock.mdb
                            └── db2
                                ├── data.mdb
                                └── lock.mdb

```

## Run training in SageMaker



可以修改的参数
```
'select_data': 'db1-db2',   # 训练的数据
'batch_ratio': '0.5-0.5',   # 数据比率
'batch_size': 160,          # batch_size.  
'num_iter': 1000,           # 训练次数
'valInterval': 100,         # 显示valid 准确率
```
更多参数请查看 `source/train.py`

In [ ]:
from sagemaker.pytorch import PyTorch
inputs = 's3://{}/{}/'.format(bucket, prefix)
print(inputs)

estimator = PyTorch(entry_point='train.py',
                    source_dir='source',   # 会复制到 /opt/ml/code/ 里面
                    role=role,
                    framework_version='1.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p3.2xlarge',
                    #train_instance_type='ml.m5.large',  # 如果没有开通GPU机型，可以使用 ml.m5.large
                    base_job_name='ocr-train',
                    train_volume_size=100,
                    train_max_run=432000,
                    output_path='s3://{}/{}/output'.format(bucket, prefix),   # 生成路径 /opt/ml/model/
                    hyperparameters={
                        'train_data': '/opt/ml/input/data/training/train',
                        'valid_data': '/opt/ml/input/data/training/valid',
                        'Transformation': 'TPS',
                        'FeatureExtraction': 'ResNet',
                        'SequenceModeling': 'BiLSTM',
                        'Prediction': 'Attn',
                        'select_data': 'db1-db2',
                        'batch_ratio': '0.5-0.5',
                        'batch_size': 16,   #GPU机型可以设置160, 具体根据选择的参数和机器的显存大小 做调整
                        'num_iter': 100,
                        'valInterval': 10,
                        
                    })
estimator.fit({'training': inputs})